# Aerial Cactus Identification

# Load Packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import pickle
import cv2
from tqdm import tqdm 
import matplotlib as mpl

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import zipfile 

# Load DataFrame

In [ ]:
train = pd.read_csv("../input/aerial-cactus-identification/train.csv", dtype=str)
print(train.shape)

In [ ]:
train.head(10)

# Extract Images

In [ ]:
zip_ref = zipfile.ZipFile('/kaggle/input/aerial-cactus-identification/train.zip')
zip_ref.extractall()

In [ ]:
train_path = "train/"
print('Training Images:', len(os.listdir(train_path)))

In [ ]:
print('Training Images:', len(os.listdir('train/')))

for i in range(10):
  img = plt.imread('train/' + train.id[i])
  print('Images shape', img.shape)

# View Sample of Images

In [ ]:
sample = train.sample(n=16).reset_index()

plt.figure(figsize=(8,8))

for i, row in sample.iterrows():

    img = mpimg.imread(f'train/{row.id}')    
    label = row.has_cactus

    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0, -5, f'Class {label}', color='k')
        
    plt.axis('off')

plt.tight_layout()
plt.show()


# Load Model

Here we load a model that has been previously trained on this dataset. 

In [ ]:
cnn = load_model('../input/models/Cactus/cactus_model_v01.h5')
cnn.summary()

# Heatmap Functions

In the cell below we create the two functions that will be used to generate the heatmaps. 
* The function `create_grad_model()` is used to create a "gradient model" from our CNN. It needs to be called only once. 
* The function `compute_heatmap()` returns a numpy array representing the heatmap for a single image. 

In [ ]:
def create_grad_model(model):
    for layer in reversed(model.layers):
        if len(layer.output_shape) == 4:
            last_conv_layer = layer.name
            break

    grad_model = tf.keras.models.Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(last_conv_layer).output, model.output])
    
    return grad_model 

def compute_heatmap(image, class_ix, grad_model):

    with tf.GradientTape() as tape:
        inputs = tf.cast(image, tf.float32)
        (conv_outputs, predictions) = grad_model(inputs)
        loss = predictions[:, class_ix]
    grads = tape.gradient(loss, conv_outputs)

    cast_conv_outputs = tf.cast(conv_outputs > 0, "float32")
    cast_grads = tf.cast(grads > 0, "float32")
    guided_grads = cast_conv_outputs * cast_grads * grads

    conv_outputs = conv_outputs[0]
    guided_grads = guided_grads[0]

    weights = tf.reduce_mean(guided_grads, axis=(0, 1))

    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs), axis=-1)

    (w, h) = (image.shape[2], image.shape[1])
    heatmap = cv2.resize(cam.numpy(), (w, h))
        
    return heatmap

# Example: First Heatmap

In [ ]:
# Create Gradient Model
gm = create_grad_model(cnn)

# Select Image and Create Heatmap
filename = train.id[0]
img = mpimg.imread(f'train/{filename}')       
tensor = img.reshape(-1,32,32,3) / 255
heatmap = compute_heatmap(tensor, 1, gm)

plt.figure(figsize=[9,3])

# Display Image
plt.subplot(1,3,1)
plt.imshow(img)
plt.axis('off')

# Display Heatmap
plt.subplot(1,3,2)
plt.imshow(heatmap, cmap='coolwarm')
plt.axis('off')

# Display Image and Heatmap Together
plt.subplot(1,3,3)
plt.imshow(img, alpha=0.8, cmap='binary_r')
plt.imshow(heatmap, alpha=0.6, cmap='coolwarm')
plt.axis('off')
    
plt.show()

The function below is used to get a sense as to the distribution of the heatmap values for images in the dataset. It creates heatmaps for every image in the DataFrame `df` and stores them into an array named values, which is then returned. 

# Displaying Multiple Heatmaps

In [ ]:
def get_heatmap_dist(df, class_ix, gm):

    values = None
    for i, row in tqdm(df.iterrows()):
        img = mpimg.imread(f'train/{row.id}')    
        tensor = img.reshape(1,32,32,3) / 255
        hm = compute_heatmap(tensor, class_ix, gm)

        if values is None:
            values = hm.flatten()
        else:
            values = np.hstack([values, hm.flatten()])

    return values

In [ ]:
values = get_heatmap_dist(train.sample(1000, random_state=1), 1, gm)

The colorscale that matplotlib assigns to an heatmap are determined be the values present within that image. As a result, each heatmap we display will have its own colorscale by default. We can make the colorscales uniform by specifying a "norm" for the color scale. This requires us to specify a lower and upper end for the color scale. We do that in the cell below. This requires some trial and error in order to get the heatmaps to display well. 

In [ ]:
low = np.quantile(values, 0.10)
high = np.quantile(values, 0.96)

norm = mpl.colors.Normalize(vmin=low, vmax=high)

print(low)
print(high)

We will now diplay several images along with their class activation heatmaps.

In [ ]:
# Select which images to display
indices = range(12)

for i in indices:  
    row = train.iloc[i,:]
    img = mpimg.imread(f'train/{row.id}')    
    label = row.has_cactus
    
    tensor = img.reshape(-1,32,32,3) / 255
    heatmap = compute_heatmap(tensor, 1, gm)

    if(label == '1'):
        print('Cactus Present')
    else:
        print('No Cactus')
    
    plt.figure(figsize=[9,3])

    plt.subplot(1,3,1)
    plt.imshow(img)
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(heatmap, cmap='coolwarm', norm=norm)
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(img, alpha=0.6, cmap='binary_r')
    plt.imshow(heatmap, alpha=0.6, cmap='coolwarm', norm=norm)
    plt.axis('off')
    
    plt.show()
